In [9]:
import numpy as np

# Modeling the motor
Write this later

## The results
We have

$$ \dot{v} = -\gamma_{1} v + \gamma_{2} u $$

where $v$ is velocity in $\frac{m}{s}$ and $u$ is input voltage in volts.
 

In [10]:
gamma_1 = 5
gamma_2 = 1

# Creating the controller

## Definitions
### Variables
 * $x$ is our system state variable velocity in $\frac{m}{s}$. This is usually a vector, but our system is first-order, so it is a scalar.
 * $y$ is our system output variable velocity in $\frac{m}{s}$ (Yes, $x$ and $y$ are the same)
 * $u$ is the voltage output from the controller in volts
 * $\sigma$ is the error integral in $\frac{m}{s} s = m$
 * $r$ is the car's target (reference) velocity
 * $u_{ref}$ and $x_{ref}$ are reference values of $u$ and $x$ derived from plant inversion
 
### Constants
 * $n$ is the system order (1 for us).
 * $A$, $B$, $C$ are all matricies that represent the car's physics
 * $k_{1}$ is the P gain of our controller. This is usually a matrix, but $x$ and $u$ are scalars so $k_{1}$ is scalar too.
 * $k_{2}$ is the I gain of our controller. This is a scalar.

## Converting the model to state-space form

In order to more easily use our control-design machinery, we shall place the model in state-space form:
$$ \dot{x} = A x + B u $$
$$ y = C x $$

In [11]:
A = -gamma_1  #This makes A a 2-D array for numpy
B = gamma_2
C = 1

## The controller equations
We shall implement PI trajectory-following control with state feedback

$$ u = u_{ref} - k_{1} (x - x_{ref}) - k_{2} \sigma $$
$$ \dot{\sigma} = y - r $$

We have no need of an estimator because we directly measure the state variable, velocity, with our speed sensor.

## Determining the gains $k_{1}$ and $k_{2}$
It can be shown that the overall system eigenvalues, which control how fast the car settles to the desired velocity, are given by the eigenvalues of $$\mathscr{A} - \mathscr{B}\mathscr{K}$$. There are $$n + 1$$ eigenvalues to place.

$\mathscr{A} = \begin{pmatrix}A & 0 \\C & 0\end{pmatrix}$ is a square matrix, $\mathscr{B} = \begin{pmatrix}B \\ 0\end{pmatrix}$ is a column vector, and 
$\mathscr{K} = \begin{pmatrix}k_{1} & k_{2}\end{pmatrix}$ is a row vector.

### Check for controllability and commandability
We can place the eigenvalues of $\mathscr{A} - \mathscr{B}\mathscr{K}$ anywhere we want provided that $A$ and $B$ are controllable and commandable, which is identical to checking if $\mathscr{A}$ and $\mathscr{B}$ are controllable.

$\mathscr{A}$ and $\mathscr{B}$ are controllable if and only if $|\mathscr{C}| \neq 0$, where $\mathscr{C}$ is a square matrix:
$$\mathscr{C} = \begin{pmatrix} B & AB & \cdots & A^{n-1}B\end{pmatrix}$$

In our case, the system is controllable and commandable, so we may continue with the design:
$$|\mathscr{C}| = |\begin{pmatrix} B \end{pmatrix}| = |B| = \gamma_{2} \neq 0$$

### Calculation of $\mathscr{K}$
Fortunately, Scipy can do the math for us if we tell it $\mathscr{A}$, $\mathscr{B}$, and where we want the poles to be (this is a key design parameter). See [the documentation for the function](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.place_poles.html).

In [12]:
from scipy.signal import place_poles

A_script = np.array([[A, 0],
                     [C, 0]])
B_script = np.array([[B],
                     [0]])

desired_poles = [1, 1.1]

results = place_poles(A_script, B_script, desired_poles)
#Results is returned as a 2-d matrix, when we want a 1-d matrix. The [0] converts it.
k_1, k_2 = K_script = results.gain_matrix[0]
actual_poles = results.computed_poles

print("k_1 =", k_1, " k_2 =", k_2)   #the gains
print(actual_poles) #where the poles actually ended up. Should be very close to our desired poles

k_1 = -7.100000000000003  k_2 = 1.100000000000003
[1.  1.1]


## Creating $u_{ref}$ and $x_{ref}$ via plant inversion
In general, the inverse plant is also given by a state-space model
$$ \dot{x_{inv}} = A_{inv} x_{inv}+ B_{inv} u_{inv} $$
$$ y_{inv} = C_{inv} x_{inv} + D_{inv} u_{inv}$$

where
 * $y_{inv} = \begin{pmatrix}u_{ref} \\ x_{ref}\end{pmatrix}$ is the output of the inverse system, which will be used in the controller. $x_{ref}$ is a vector the same size as $x$.
 * $x_{inv}$ is NOT $x_{ref}$. It is some artificial state used in the system.
 * $u_{inv} = \begin{pmatrix}r \\ \dot{r} \\ \vdots \\ r^{(\rho)}\end{pmatrix}$, the input to the inverse system, is derivatives of the reference signal.

and our job is to determine $A_{inv}$, $B_{inv}$, $C_{inv}$, and $D_{inv}$.
 
**Do not be confused!** $y_{inv}$, $x_{inv}$, and $u_{inv}$ are not the same as $y$, $x$, or $u$.

### Determine the relative degree of the plant, $\rho$
The relative degree is $\rho =$ Number Plant Poles $-$ Number Plant Zeros, where $1 \leq \rho \leq n$ (provided plant is controllable and commandable, as we already showed). For us, $n = 1$, so $1 \leq \rho \leq 1$, or $\rho = 1$.

In [13]:
system_order = 1

### Variable change to normal form
Our goal is to find two matricies $M$ and $N$. M is given explicitly, while N is defined implicitly (N is not unique) by satisfying the following properties:
 * $M = \begin{pmatrix}CA^{0} \\ CA^{1} \\ \vdots \\ CA^{\rho - 1}\end{pmatrix}$
 * $S = \begin{pmatrix}M \\ N\end{pmatrix}$ is square
 * $|S| \neq 0$ (So S is invertible and can be used to change variables).

#### For our system
$$M = \begin{pmatrix} CA^{0} \end{pmatrix} = C = 1$$

and $N$ has no size at all:
$$S = \begin{pmatrix} M \end{pmatrix} = 1$$

### Inverse variable change
Using the $S$ found above, find

$$S^{-1} = \begin{pmatrix}Q & R\end{pmatrix}$$

where $Q$ is a $n \times \rho$ matrix and $R$ is a $n \times (n -\rho)$ matrix.

#### For our system
$n = \rho = 1$, so $R$ is a $1 \times 0$ matrix and 
$$S^{-1} = 1 = \begin{pmatrix}Q & R\end{pmatrix} = \begin{pmatrix} Q \end{pmatrix}$$

In [14]:
Q = 1

### Putting it all together: 
Because $N$ and $R$ do not exist in our case, $A_{inv}$, $B_{inv}$, $C_{inv}$ do not exist and the inverse plant has no state variables $x_{inv}$. The model becomes
$$ \begin{pmatrix}u_{ref} \\ x_{ref}\end{pmatrix} = y_{inv} = D_{inv} u_{inv} = D_{inv} \begin{pmatrix}r \\ \dot{r}\end{pmatrix}$$

Using the formula for $D_{inv}$,
$$\gamma_{inv} = \frac{1}{C A^{\rho-1} B} = \frac{1}{C B}$$
$$D_{inv} = \begin{pmatrix} -\gamma_{inv} C A^{\rho} Q & \gamma_{inv} \\ Q & 0\end{pmatrix} = \begin{pmatrix} -\gamma_{inv} C A Q & \gamma_{inv} \\ Q & 0\end{pmatrix}$$

Unpacking the matrix, we find

$ u_{ref} = (-\gamma_{inv} C A Q) r + (\gamma_{inv}) \dot{r} =$ k_inv_r_to_u $r + $ k_inv_r_dot_to_u $\dot{r}$

$ x_{ref} = Q r =$ k_inv_r_to_x $r$



In [17]:
#BEWARE: Python uses @ for matrix multiplication. We use * here because the matricies are 1-D
gamma_inv = 1/(C * B)

k_inv_r_to_u = -gamma_inv * C * A * Q
k_inv_r_dot_to_u = gamma_inv
k_inv_r_to_x = Q

print(k_inv_r_to_u)
print(k_inv_r_dot_to_u)
print(k_inv_r_to_x)

5.0
1.0
1
